In [2]:
import mlflow
import pandas as pd

from mlflow import MlflowClient
from mlflow.exceptions import MlflowException


from src.config.config import load_catalog

d:\desarrollos\End2end\END2END\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
client = MlflowClient()
widget = 'train'

2026/02/18 12:04:27 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.schemas
2026/02/18 12:04:27 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.tables
2026/02/18 12:04:27 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.types
2026/02/18 12:04:27 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.constraints
2026/02/18 12:04:27 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.defaults
2026/02/18 12:04:27 INFO alembic.runtime.plugins: setup plugin alembic.autogenerate.comments
2026/02/18 12:04:29 INFO alembic.runtime.migration: Context impl SQLiteImpl.
2026/02/18 12:04:29 INFO alembic.runtime.migration: Will assume non-transactional DDL.


In [7]:
catalog, params = load_catalog(env="base")

nombre_run = params['nombre_run']
nombre_experimento = params['nombre_experiment']
alias = params['aliases_champion']
params_modeling = params['modeling']
nombre_usecase = params['nombre_usecase']

# features
num_features = params['num_features']
cat_features = params['cat_features']

try:
    client.get_registered_model(nombre_usecase)
    print("El modelo ya existe")
except MlflowException:
    client.create_registered_model(nombre_usecase)
    print("Modelo registrado creado.")

# Obtener mejores modelos
top_2 = mlflow.search_runs(
    experiment_names=[nombre_experimento],
    order_by=["metrics.mean_cv_accuracy DESC"],#--------Filtrar por fecha tambien---------------#
    max_results=2
)

best_run_id = top_2.iloc[0]["run_id"]
second_best_run_id = top_2.iloc[1]["run_id"]

if widget == "train":
    
    version = mlflow.register_model(f'runs:/{best_run_id}/model',nombre_usecase)

    client.set_registered_model_alias(
        name=nombre_usecase,
        alias="champion",
        version=version.version
    )

    version2 = mlflow.register_model(f'runs:/{second_best_run_id}/model',nombre_usecase)

    client.set_registered_model_alias(
        name=nombre_usecase,
        alias="challenger",
        version=version2.version
    )

elif widget == "retraining":
    version = mlflow.register_model(f'runs:/{best_run_id}/model',nombre_usecase)

    client.set_registered_model_alias(
        name=nombre_usecase,
        alias="challenger",
        version=version.version
    )

else:
    raise ValueError("Widget debe ser 'train' o 'retraining'")





Registered model 'titanic' already exists. Creating a new version of this model...
2026/02/18 12:16:17 WARNING mlflow.tracking._model_registry.fluent: Run with id 3c8718753ee341749580bc7bc60c6e01 has no artifacts at artifact path 'model', registering model based on models:/m-6dfea2fbeda04791b80d68ecf4fa13db instead


El modelo ya existe


Created version '1' of model 'titanic'.
Registered model 'titanic' already exists. Creating a new version of this model...
2026/02/18 12:16:18 WARNING mlflow.tracking._model_registry.fluent: Run with id 155c068e4bcb4937b87abfbcffadb43f has no artifacts at artifact path 'model', registering model based on models:/m-dfc906704bd94272a2e72f3a256df7ff instead
Created version '2' of model 'titanic'.
